In [17]:
import os
import numpy as np
import pandas as pd


import plotly.graph_objects as go
from plotly.graph_objects import Figure
from plotly.graph_objects import Scatter

from  sxo.util.runtime.timeseries import RedisTs

In [18]:
os.environ["REDIS_PASS"] = "boss"

In [28]:
ts_bid = RedisTs('ts_bid_37374')
ts_ask = RedisTs('ts_ask_37374')
ts_bsz = RedisTs('ts_bsz_37374')
ts_asz = RedisTs('ts_asz_37374')
bid = ts_bid.get_range(name='bid')
ask = ts_ask.get_range(name='ask')
bsz = ts_bid.get_range(name='bsz')
asz = ts_ask.get_range(name='asz')
bid_ask = bid.join(ask, how='inner').join(bsz, how='inner').join(asz, how='inner').reset_index()
bid_ask['t'] = bid_ask['t'].values.astype('datetime64[ms]')

df_plt = bid_ask.copy()[bid_ask.t > np.datetime64('2023-06-19 00:00:00')]
m = 15
fig = Figure(layout=go.Layout(height=500, margin={'t':m,'b':m,'l':m,'r':m}))
fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['ask'], name='ask', mode="markers", marker={'size':4},showlegend=False))
fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['bid'], name='bid', mode="markers", marker={'size':4},showlegend=False))
fig.show()

In [68]:
def make_bin(x):
    #df = pd.DataFrame()
    df = pd.Series()
    t = x['t'].values
    bid, ask, bsz, asz = x.bid.values, x.ask.values, x.bsz.values, x.asz.values
    bin_end = x['bin_end'].values[0]
    mid  = (bid + ask)/2.0
    dt = np.diff(np.append(t, bin_end))

    df['bin_end'] = bin_end 
    df['o'] = mid[0]
    df['h'] = np.max(mid)
    df['l'] = np.min(mid)
    df['c'] = mid[-1]
    df['twap'] = np.sum(mid*dt)/np.sum(dt)

    return df

def gen_binned_data(ticks:pd.DataFrame,
                    bin_size_sec:int,
                    time_col:str = "t",
                    ) -> pd.DataFrame:

    ticks['bin_start'] = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[s]')
    ticks['bin_end'] = ticks['bin_start'].values + np.timedelta64(bin_size_sec, 's')

    padded = ticks.groupby(['bin_start'], as_index=False).apply(make_bin).reset_index()
    return padded

x = gen_binned_data(bid_ask.copy(), 60)
x

,index,bin_start,bin_end,o,h,l,c,twap
0,0,2023-06-06 19:20:00,2023-06-06 19:30:00,1.162185,1.162205,1.162165,1.162185,1.162136
1,1,2023-06-06 20:20:00,2023-06-06 20:30:00,1.161995,1.162055,1.161985,1.162045,1.161919
2,2,2023-06-06 20:30:00,2023-06-06 20:40:00,1.162045,1.162115,1.161965,1.161975,1.161967
3,3,2023-06-06 20:40:00,2023-06-06 20:50:00,1.161965,1.162005,1.161845,1.161905,1.161799
4,4,2023-06-06 20:50:00,2023-06-06 21:00:00,1.161895,1.161895,1.161745,1.161855,1.161785
...,...,...,...,...,...,...,...,...
634,634,2023-06-19 17:20:00,2023-06-19 17:30:00,1.170565,1.170565,1.170435,1.170465,1.170472
635,635,2023-06-19 17:30:00,2023-06-19 17:40:00,1.170455,1.170505,1.170295,1.170405,1.170360
636,636,2023-06-19 17:40:00,2023-06-19 17:50:00,1.170395,1.170655,1.170325,1.170485,1.170467
637,637,2023-06-19 17:50:00,2023-06-19 18:00:00,1.170455,1.170585,1.170455,1.170565,1.170457


In [26]:
# def pad_bin(x:pd.DataFrame) -> pd.DataFrame:
#     times = x['t'].values
#     bin_times = x['bin_start'].values
#     if times[0] != bin_times[0]:
#         x =
#     return x

xx = x[x.bin_start==np.datetime64('2023-06-13 15:23:00')]

#pad_bin(xx)